In [19]:
import pandas as pd
import sys
import numpy as np
sys.path.append('../../')
# get repo root actual dir (for file reading)
from scripts.system_helpers.repo_root import REPO_ROOT
from scripts.data_helpers.elexon_api_helpers import call_physbm_api
from pyproj import Transformer

config = {'start_date' : '01/01/2025',
          'end_date' : '01/01/2025',
          'repd_name' : 'doronell',
          'elexon_id' : ['T_DOREW-1','T_DOREW-2']}

start_date = config['start_date']
end_date = config['end_date']
repd_name = config['repd_name']
elexon_id = config['elexon_id']

met_ss_api = 'eyJ4NXQjUzI1NiI6Ik5XVTVZakUxTkRjeVl6a3hZbUl4TkdSaFpqSmpOV1l6T1dGaE9XWXpNMk0yTWpRek5USm1OVEE0TXpOaU9EaG1NVFJqWVdNellXUm1ZalUyTTJJeVpBPT0iLCJraWQiOiJnYXRld2F5X2NlcnRpZmljYXRlX2FsaWFzIiwidHlwIjoiSldUIiwiYWxnIjoiUlMyNTYifQ==.eyJzdWIiOiJGYXJxdWhhcnNvbmFyQGdtYWlsLmNvbUBjYXJib24uc3VwZXIiLCJhcHBsaWNhdGlvbiI6eyJvd25lciI6IkZhcnF1aGFyc29uYXJAZ21haWwuY29tIiwidGllclF1b3RhVHlwZSI6bnVsbCwidGllciI6IlVubGltaXRlZCIsIm5hbWUiOiJzaXRlX3NwZWNpZmljLTkwMDEzZmUxLWQ1YmUtNDFjOC04YWJiLTQ1Zjc2ZjI1MTdkZCIsImlkIjoyNTQ2MywidXVpZCI6ImRhMjBhMWNiLWVhOTItNDEzNS1hOGI0LTJkMDYxNDNjYzQwYSJ9LCJpc3MiOiJodHRwczpcL1wvYXBpLW1hbmFnZXIuYXBpLW1hbmFnZW1lbnQubWV0b2ZmaWNlLmNsb3VkOjQ0M1wvb2F1dGgyXC90b2tlbiIsInRpZXJJbmZvIjp7IndkaF9zaXRlX3NwZWNpZmljX2ZyZWUiOnsidGllclF1b3RhVHlwZSI6InJlcXVlc3RDb3VudCIsImdyYXBoUUxNYXhDb21wbGV4aXR5IjowLCJncmFwaFFMTWF4RGVwdGgiOjAsInN0b3BPblF1b3RhUmVhY2giOnRydWUsInNwaWtlQXJyZXN0TGltaXQiOjAsInNwaWtlQXJyZXN0VW5pdCI6InNlYyJ9fSwia2V5dHlwZSI6IlBST0RVQ1RJT04iLCJzdWJzY3JpYmVkQVBJcyI6W3sic3Vic2NyaWJlclRlbmFudERvbWFpbiI6ImNhcmJvbi5zdXBlciIsIm5hbWUiOiJTaXRlU3BlY2lmaWNGb3JlY2FzdCIsImNvbnRleHQiOiJcL3NpdGVzcGVjaWZpY1wvdjAiLCJwdWJsaXNoZXIiOiJKYWd1YXJfQ0kiLCJ2ZXJzaW9uIjoidjAiLCJzdWJzY3JpcHRpb25UaWVyIjoid2RoX3NpdGVfc3BlY2lmaWNfZnJlZSJ9XSwidG9rZW5fdHlwZSI6ImFwaUtleSIsImlhdCI6MTc1ODEyMDY3MywianRpIjoiNDIzMWE4N2QtNDhhNC00ZDAxLTg3YjctMmY5MWYwOTkwNTBiIn0=.PuBtkHdKKohOO9P2NuiS0r9KMAfnosx1w8kY1SvPZnS0-t_iA290CwlBOvI1L8dwl51jftt2k8pF6lqDQveFl4yQ2b_SZ9zjBy1nshSppnHO1HBZSgFgneat03-ueOettyZc5VQB425Zj0zqi7e01VAFaGMu8pmVYAJDKNFipg-lrgmXumgyOe1OtrLjc6JzPfNKk0hjDB4WUy_ds1XJ79YeucrQuyR507fTc8HF5wxMe5do7na26fboic3E5XdePNDPj_-i1vle-7nqCXwEn58OE75b2EwPDAtAhw5ntdxuzexXVeATOgjhpbCbsp3kDXWIWdLoUXidwiAxy7LW6g=='

In [11]:
# wind farm of choice given that there are two but they are in the same location
# Dorenell Wind Farm
# 1. get locational data for Dorenell Wind Farm
df_loc = pd.read_csv(f'{REPO_ROOT}/data/{repd_name}_repd.csv')
df_loc[['Site Name', 'Address', 'Country', 'Post Code',	'X-coordinate',	'Y-coordinate']]

display(df_loc[['Site Name', 'Address', 'Country', 'Post Code',	'X-coordinate',	'Y-coordinate']])
# Transformer from OSGB36 / British National Grid → WGS84 standard GPS
transformer = Transformer.from_crs("EPSG:27700", "EPSG:4326", always_xy=True)
x, y = df_loc['X-coordinate'].iloc[0], df_loc['X-coordinate'].iloc[0]
lon, lat = transformer.transform(x, y)
print(lon, lat)

,Site Name,Address,Country,Post Code,X-coordinate,Y-coordinate
0,Dorenell Wind Farm (Previously Site A and B Sc...,"Glenlivet Estate, Dufftown, Speyside, Moray",Scotland,NaN,332000,829500


-3.011912044445406 52.8811455063142


In [7]:
# 2. get PN data of doronell
data_df = call_physbm_api(start_date, end_date)
data_df=data_df.sort_values(['recordType','bmUnitID','timeFrom'])
data_df = data_df[data_df['bmUnitID'].isin(elexon_id)]
data_df

INFO:scripts.data_helpers.elexon_api_helpers:Calling BOAS API for None
INFO:scripts.data_helpers.elexon_api_helpers:Getting PN from 2025-01-01 00:00:00
INFO:scripts.data_helpers.elexon_api_helpers:Getting BOALF from 2025-01-01 00:00:00


,recordType,settlementDate,settlementPeriod,timeFrom,timeTo,levelFrom,levelTo,Accept ID,acceptanceTime,deemedBidOfferFlag,soFlag,rrScheduleFlag,bmUnitID,local_datetime
1864,PN,2024-12-31,48,2024-12-31T23:30:00Z,2025-01-01T00:00:00Z,96,96,NaN,NaN,NaN,NaN,NaN,T_DOREW-1,2024-12-31 23:30:00+00:00
1865,PN,2024-12-31,48,2024-12-31T23:30:00Z,2025-01-01T00:00:00Z,81,80,NaN,NaN,NaN,NaN,NaN,T_DOREW-2,2024-12-31 23:30:00+00:00


In [ ]:
# 3. get weather data of doronell
from scripts.data_helpers.metoffice_api_helpers import met_office_forecast_api 
# attempt 1 metoffice - only gets current and next few days data
requestHeaders = {"apikey": met_ss_api}
headers = {'accept': "application/json"}
headers.update(requestHeaders)
base_url = "https://data.hub.api.metoffice.gov.uk/sitespecific/v0/point/"
timesteps = 'three-hourly'
latitude = lat
longitude = lon
req = met_office_forecast_api(baseUrl = base_url, 
                  timesteps = timesteps, 
                  requestHeaders = requestHeaders, 
                  latitude = latitude, 
                  longitude = longitude, 
                  excludeMetadata = False, 
                  includeLocation = True)
df_forecast = pd.DataFrame(req.json()['features'][0]['properties']['timeSeries'])
df_forecast[['time','windSpeed10m',	'windDirectionFrom10m',	'windGustSpeed10m',	'max10mWindGust']].head()

{"type":"FeatureCollection","features":[{"type":"Feature","geometry":{"type":"Point","coordinates":[-3.0551,52.8569,126.0]},"properties":{"location":{"name":"Oswestry"},"requestPointDistance":3959.8192,"modelRunDate":"2025-09-18T15:00Z","timeSeries":[{"time":"2025-09-18T15:00Z","maxScreenAirTemp":18.79,"minScreenAirTemp":18.25,"max10mWindGust":12.86,"significantWeatherCode":8,"totalPrecipAmount":0.0,"totalSnowAmount":0,"windSpeed10m":7.0,"windDirectionFrom10m":227,"windGustSpeed10m":10.8,"visibility":18608,"mslp":101590,"screenRelativeHumidity":85.78,"feelsLikeTemp":16.2,"uvIndex":2,"probOfPrecipitation":21,"probOfSnow":0,"probOfHeavySnow":0,"probOfRain":21,"probOfHeavyRain":9,"probOfHail":1,"probOfSferics":1},{"time":"2025-09-18T18:00Z","maxScreenAirTemp":18.71,"minScreenAirTemp":17.77,"max10mWindGust":12.77,"significantWeatherCode":7,"totalPrecipAmount":0.0,"totalSnowAmount":0,"windSpeed10m":4.73,"windDirectionFrom10m":221,"windGustSpeed10m":11.92,"visibility":22300,"mslp":101561,"sc

,time,windSpeed10m,windDirectionFrom10m,windGustSpeed10m,max10mWindGust
0,2025-09-18T15:00Z,7.00,227,10.80,12.86
1,2025-09-18T18:00Z,4.73,221,11.92,12.77
2,2025-09-18T21:00Z,4.41,215,11.37,12.11
3,2025-09-19T00:00Z,5.83,211,13.30,14.33
4,2025-09-19T03:00Z,5.54,208,12.65,14.31


In [16]:
# try hasgrid data - historic but only get yearly or seasonal average
import xarray as xr
# Open the file
hasgrid_xr = xr.open_dataset(f'{REPO_ROOT}/data/sfcWind_hadukgrid_uk_1km_ann_202301-202312.nc')
print(list(hasgrid_xr.variables))

['sfcWind', 'transverse_mercator', 'time_bnds', 'projection_y_coordinate_bnds', 'projection_x_coordinate_bnds', 'time', 'projection_y_coordinate', 'projection_x_coordinate', 'latitude', 'longitude']


In [25]:
df_hasgrid_wind = hasgrid_xr['sfcWind']
df_hasgrid_wind = df_hasgrid_wind.to_dataframe().reset_index()

In [26]:
df_hasgrid_wind[(np.round(df_hasgrid_wind['longitude'],3) == -3.012)]

,time,projection_y_coordinate,projection_x_coordinate,latitude,longitude,sfcWind
55425,2023-07-01,-138500.0,325500.0,48.649187,-3.011578,NaN
56325,2023-07-01,-137500.0,325500.0,48.658182,-3.011758,NaN
57225,2023-07-01,-136500.0,325500.0,48.667178,-3.011938,NaN
58125,2023-07-01,-135500.0,325500.0,48.676173,-3.012118,NaN
59025,2023-07-01,-134500.0,325500.0,48.685169,-3.012298,NaN
...,...,...,...,...,...,...
1189443,2023-07-01,1121500.0,343500.0,59.975041,-3.012313,NaN
1246144,2023-07-01,1184500.0,344500.0,60.540814,-3.011703,NaN
1247044,2023-07-01,1185500.0,344500.0,60.549792,-3.011984,NaN
1247944,2023-07-01,1186500.0,344500.0,60.558770,-3.012264,NaN


In [30]:
# open weather data - not allowed either witout paying
import requests
lat = 3.01
lon = 52.88
start = 1369728000
end = 1369729000
open_weather_data_api_key = '1548fbeeba9526658fd7bf8ba300189b'

open_weather_api_url = f'https://history.openweathermap.org/data/2.5/history/city?lat={lat}&lon={lon}&type=hour&start={start}&end={end}&appid={open_weather_data_api_key}'
df = requests.get(open_weather_api_url)

In [33]:
# ECMWF data
import xarray as xr

# Open GRIB file
ds = xr.open_dataset(f'{REPO_ROOT}/data/data.grib', engine="cfgrib")


ValueError: unrecognized engine 'cfgrib' must be one of your download engines: ['netcdf4', 'scipy', 'store']. To install additional dependencies, see:
https://docs.xarray.dev/en/stable/user-guide/io.html 
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html